In [1]:
#!connect signalr --kernel-name rover --hub-url http://10.0.0.252:1024/kernelhub

In [1]:
#!rover
#!reset --all


In [1]:
#!rover
Microsoft.DotNet.Interactive.Formatting.Formatter.ListExpansionLimit = 42;
ResourceScanner.CaptureFromCamera(RoverBody.Camera);
ResourceScanner.UseUri(new Uri("38100/predict/c67df0e6-47a3-4d1c-973e-890bf3087a0e"));
//ResourceScanner.UseModel(new DirectoryInfo("/home/pi/models/pics"));

RoverBody.Reset();

In [1]:
#!rover
var testSector = new CameraSector(Angle.FromDegrees(-15), Angle.FromDegrees(-50));
RoverBody.TiltController.GoToSector(testSector);
Task.Delay(500).Wait();
testSector.CapturedFrame = RoverBody.Camera.GetFrame().Focus();
testSector.ClassificationResults = ResourceScanner.AnalyseFrame(testSector.CapturedFrame);
testSector

In [1]:
#!rover
var testSector = new CameraSector(Angle.FromDegrees(0), Angle.FromDegrees(0));
RoverBody.TiltController.GoToSector(testSector);
Task.Delay(500).Wait();
testSector.CapturedFrame = RoverBody.Camera.GetFrame().Focus();
testSector.ClassificationResults = ResourceScanner.AnalyseFrame(testSector.CapturedFrame);
testSector

In [1]:
#!rover

var scannedSectors = CameraSector
.CreateSectors(5,5, Angle.FromDegrees(-60),Angle.FromDegrees(60),Angle.FromDegrees(-15),Angle.FromDegrees(30))
.Distinct()
.ToArray();
CameraSector currentSector = null;

In [1]:
#!rover

bool IsEnergyCard(ClassificationResults result)  => 
    result != null && !result.Prediction.Label.Contains("no energy");

bool AllSectorScanned(IEnumerable<CameraSector> sectors) => 
    scannedSectors.All(v => v.Marked);

bool EnoughEnergyCardsFound(IEnumerable<CameraSector> sectors, int requiredCount) =>
    sectors?.Where(s => IsEnergyCard(s.ClassificationResults))
           .Select(s => s.ClassificationResults.Prediction.Label)
           .Distinct()
           .Count() >= requiredCount;

bool AllSectorScannedOrEnoughResourceFound(IEnumerable<CameraSector> sectors, int requiredCount) =>
    AllSectorScanned(sectors) || EnoughResourcesFound(sectors, requiredCount);

bool AnalyseSectorForEnergyCards(CameraSector sector) {
    if (sector?.CapturedFrame != null) {        
        sector.ClassificationResults = ResourceScanner.AnalyseFrame(sector.CapturedFrame);               
        return IsEnergyCard(sector.ClassificationResults);
    }
    
    return false;
}

Reset the state, we track sectors being scanned and the resoruce found

In [1]:
#!rover

RoverBrain.Perceive = () => {
    if (currentSector == null){
        return;
    }

    if (currentSector.CapturedFrame == null) {
        Task.Delay(500).Wait();
        currentSector.CapturedFrame = RoverBody.Camera.GetFrame().Focus();
    }
};

In [1]:
#!rover
RoverBody.Reset();

In [1]:
#!rover
RoverBrain.ClearState = () => {
    foreach (var sector in scannedSectors) {
        sector.Reset();
    }

    currentSector = null;
    RoverBody.Reset();
};

Let's capture the current sector frame

In [1]:
#!rover

RoverBrain.Act = () => { 
    if(currentSector != null){
        RoverBody.TiltController.GoToSector(currentSector);
    }   
};

Define the planning logic:
1. Terminate when all sector have been visited and image was aquired
2. Classify the current sector if needed
3. Blink lights if a classification has been aquired

In [1]:
#!rover

RoverBrain.Plan = () => {
    if (AllSectorScanned(scannedSectors)){
        RoverBody.AllLightsOff();
        return PlanningResult.NoPlan;
    }  

    if (currentSector != null) {
       
        currentSector.Marked = true;
    }   
    
    currentSector = scannedSectors.FirstOrDefault(s => s.Marked == false);
    
    RoverBody.AllLightsOff();

    return currentSector != null 
        ? PlanningResult.NewPlan 
        : PlanningResult.NoPlan;    
};

In [1]:
#!rover

RoverBrain.Plan = () => {
    if (AllSectorScanned(scannedSectors)){
        RoverBody.AllLightsOff();
        return PlanningResult.NoPlan;
    }  

    if (currentSector != null) {
       
        currentSector.Marked = true;
    }

    if (AnalyseSectorForEnergyCards(currentSector))
    {
        RoverBody.BlinkAllLights();
    }else {
        RoverBody.AllLightsOff();
    }
    
    currentSector = scannedSectors.FirstOrDefault(s => s.Marked == false);

    RoverBody.AllLightsOff();

    return currentSector != null 
        ? PlanningResult.NewPlan 
        : PlanningResult.NoPlan;  
};

In [1]:
#!rover

RoverBrain.Plan = () => {

    if (AllSectorScanned(scannedSectors) || EnoughResourcesFound(scannedSectors, 3)){
        RoverBody.AllLightsOff();
        return PlanningResult.NoPlan;
    }  

    if (currentSector != null) {       
        currentSector.Marked = true;
    }
   
    if (AnalyseSectorForResource(currentSector))
    {
        RoverBody.BlinkAllLights();
    }else {
        RoverBody.AllLightsOff();
    }
    
    currentSector = scannedSectors.FirstOrDefault(s => s.Marked == false);

    RoverBody.AllLightsOff();

    return currentSector != null 
        ? PlanningResult.NewPlan 
        : PlanningResult.NoPlan;  
};

Let's move the camera to the sector we planned to scan

In [1]:
#!rover
RoverBrain.ClearState();

Let's explore the sectors

In [1]:
#!rover
scannedSectors
    .Where(s => s.ClassificationResults != null )
    .GroupBy(s => s.ClassificationResults.Prediction.Label)
    .Select(g => new { Label = g.Key, Count = g.Count()})


In [1]:
#!rover
scannedSectors.Count(s => s.Marked == true)

In [1]:
#!rover
scannedSectors.Where(s => s.Marked == true)

In [1]:
#!rover
scannedSectors

In [1]:
#!rover
currentSector

In [1]:
#!rover
testSector

In [1]:
#!rover
AllSectorScanned(scannedSectors)

In [1]:
#!rover
currentSector = scannedSectors.FirstOrDefault(s => s.Marked == false);